In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_train = pd.read_csv('C:\\Users\\v.behl\\Downloads\\sentiment140\\training.1600000.processed.noemoticon.csv' , encoding='latin-1') ###replace the file and address of the  df
df_train.head()
cols=["target","id","date","status","author","Contents"]
df_train.columns=cols
df_train = df_train.sample(frac=.1).reset_index(drop=True)

In [3]:
#df_train.target = df_train.target.replace(4,1)
df_train.target.value_counts()

0    80119
4    79881
Name: target, dtype: int64

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\v.behl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
def text_clean(col1):
    col2 = col1.apply((lambda x: re.sub(r'http\S+','', x,flags=re.MULTILINE)))
    col2 = col2.apply((lambda x: re.sub(r'@\S+','', x,flags=re.MULTILINE)))
    col2 = col2.apply(lambda x: x.lower())
    col2 = col2.apply((lambda x: re.sub('[^a-zA-z0-9\s]',' ',x)))
    col2 = col2.apply((lambda x: re.sub(r'\brt\b','',x)))
    #pattern = re.compile(r'\b(' + r'|'.join(stop) + r')\b\s*')
    #col2 = col2.apply((lambda x: re.sub('', x)))
    col2 = col2.apply((lambda x: re.sub(' +',' ',x)))
    col2 = col2.apply(lambda x: x.lstrip())
    return col2

df_train.loc[:,'Content_updated'] = text_clean(df_train['Contents'])

In [27]:
list(df_train.Contents[0:1000])

['@xjonberg my phone keeps messing up  fixxxx it ! LOL',
 "@mileycyrus aw, i'm sorry. but that was really cute. ",
 '@agentdelblaine yeahh ',
 "@lewk screw you ! I've gone to bed to sulk ",
 '@JonathanRKnight Jon, IF your still on, tell Donnie he OFFICIALLY has OVER 60K followers on his page. ',
 '@stewartcink At least you are on Twitter and connecting with the fans! ',
 "Waiting outside for @elizabethistiny so we can go to Panera! Haven't been in like 2 weeks! I've missed it ",
 'watching garfield 2 - i miss uk soo much ',
 '@birbrilo coffee still sucks though ',
 "@jenniferhaymore No, I'm just sending chocolate to #vano winners this week.  Have never sent choc with arcs, virtual or otherwise. ",
 'nope still not working?? feels like im letting misha and the minions down   lol will keep trying though,well until 1am ',
 "Love @tweetdeck for iPhone, especially the syncing, but it's crashed on me several times within the past few minutes ",
 'Making pasta and insalata blu with mom ',
 '@

In [5]:
# split the dataset into training and validation datasets 
from sklearn.model_selection import train_test_split
from scipy import sparse

#Training and testing split - with a split of 70:30
train_x, valid_x, train_y, valid_y = train_test_split(df_train['Content_updated'],df_train['target'], test_size=0.3, random_state=123)

#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['Content_updated'], df_train['target'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [6]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_train['Content_updated'])
# import pickle
# with open('C:\\Users\\tapojyoti.paul\\tokenizer_sentiment.pickle', 'wb') as handle:
#     pickle.dump(count_vect, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [15]:
vectorizer = TfidfVectorizer( analyzer = "word" ,use_idf= True , ngram_range= (1,2), min_df= 5)

X_train_feat = vectorizer.fit_transform(train_x)

X_test_feat = vectorizer.transform(valid_x)

In [17]:
X_train_feat.shape

(112000, 41655)

In [43]:
features = vectorizer.fit_transform(train_x)

In [47]:
# df_train['target'] = df_train['Product'].factorize()[0]
# target_df_train = df_train[['Content', 'target']].drop_duplicates().sort_values('target')
# category_to_id = dict(target_df_train.values)
# id_to_category = dict(target_df_train[['target', 'Product']].values)
# df_train.head()

target          id                          date    status          author  \
0       0  1989365235  Mon Jun 01 01:16:18 PDT 2009  NO_QUERY   mayhemstudios   
1       1  2050893958  Fri Jun 05 19:29:09 PDT 2009  NO_QUERY  ymarisatambien   
2       2  1960374271  Fri May 29 07:51:47 PDT 2009  NO_QUERY          missxo   
3       3  1686461364  Sun May 03 05:32:09 PDT 2009  NO_QUERY      ycgonzalez   
4       4  1968914963  Fri May 29 22:32:05 PDT 2009  NO_QUERY   MaanElizabeth   

                                            Contents  \
0                     @Yooner sorry, I'm not a dev.    
1  Ok so I can't wait to get home and hit my pill...   
2  Just back home from a little gathering with so...   
3                         upset stomach = no hiking    
4  @ChariceManiacs  Goodnighttttttttttt! Sweetdre...   

                                     Content_updated  
0                               sorry i m not a dev   
1  ok so i can t wait to get home and hit my pill...  
2  just back home from a little gathering with so...  
3                           upset stomach no hiking   
4                   goodnighttttttttttt sweetdreams

In [50]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(df_train.target.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

NameError: name 'labels' is not defined

In [38]:
tfidf_tags = []
top_n = -5
feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(features)
for i, e in enumerate(tfidf_sorting):
    tmp_tags = []
    indexes = e[top_n:]
    for idx in indexes:
        cur_tag = feature_array[idx]
        if features[i][idx] > 0.1 and len(cur_tag)>3 and '_' not in cur_tag:
            tmp_tags.append(cur_tag.replace(' ', '-'))
    tfidf_tags.append(",".join(tmp_tags))

IndexError: invalid index to scalar variable.

In [18]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    prob_score = classifier.predict_proba(feature_vector_valid)[:,1]
    
    return [metrics.accuracy_score(predictions, valid_y),classifier,prob_score,predictions]

In [28]:
svc = SVC(kernel= 'linear',random_state=123, C=1, probability=True)
# model=svc.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# y_pred_proba = model.predict_proba(X_test)[:, 1]
# [fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
# print('Train/Test split results:')
# print(svc.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
# print(svc.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
# print(svc.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

In [19]:
from xgboost.sklearn import XGBClassifier 

xgb_sentiment = XGBClassifier(
 learning_rate =0.01,
 n_estimators=100,
 max_depth=10,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 select=0)

# Extereme Gradient Boosting on Count Vectors
all = train_model(xgb_sentiment, X_train_feat.tocsc(), train_y, X_test_feat.tocsc())
print ("Xgb, Count Vectors: ", all[0])

    

Xgb, Count Vectors:  0.668


In [ ]:
train_model(svc,X_train_feat, train_y,X_test_feat)

In [12]:
print(all)

[0.628625, XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=6, missing=None, n_estimators=100, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=27,
              select=0, silent=None, subsample=0.8, verbosity=1), array([0.5060536 , 0.29151335, 0.5883897 , ..., 0.5060536 , 0.6483882 ,
       0.5060536 ], dtype=float32), array([1, 0, 1, ..., 1, 1, 1], dtype=int64)]


In [60]:
final_df= pd.read_csv("test_airbnb_comments.csv")

In [61]:
classifier_xgb= all[1]
import pickle
# saving
with open('C:\\Users\\v.behl\\OneDrive - Accenture\\030_AirBnB\\xgb_sentiment_classifier_1_27_18.pickle', 'wb') as handle:
     pickle.dump(classifier_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)
######################################################################
final_df.loc[:,'Content_updated'] = text_clean(final_df['contents'])
# transform the training and validation data using count vectorizer object
X_valid =  vectorizer.transform(final_df['Content_updated'])
classifier=all[1]

In [62]:
predictions = encoder.inverse_transform(classifier_xgb.predict(X_valid))    
prob_score = classifier_xgb.predict_proba(X_valid)[:,1]
final_df["Sentiment"] = predictions
final_df["Probability"]=prob_score
final_df.columns
final_df_copy = final_df


final_df.loc[final_df.Probability <= .25, 'sentiment_fin'] = 'Negative' 
final_df.loc[(final_df.Probability > .25) & (final_df.Probability < .6) , 'sentiment_fin'] = 'Neutral' 
final_df.loc[final_df.Probability >= .6, 'sentiment_fin'] = 'Positive' 
#final_df.loc[(final_df.Author.str.contains(brandname) == True) & (final_df.sentiment_fin=="Negative") , 'sentiment_fin'] = 'Neutral' 

final_df.groupby(['sentiment_fin']).size()
final_df.to_csv("airbnb_sentiment_test_v1.csv", sep=',',index = False, encoding='utf-8')         


In [51]:
string = "AirBnB customer service is a complete wreck.  They are unprofessional, they give an endless ran around and never solve anything.  The telephone support does not know anything at all about anything worth dialing the phone for.  There is not way to get in touch with a manager over the phone.  The most their 'managers' do is type a message on the platform but will not speak to you on then phone.  They simply type some ******** and close the case.  Even through the messaging platform they will take days to respond, then reply with 'I was out of the office', and close the case within hours!  It's not only bad customer service.  They have no customer service.  Their hosts are getting fed up with them.   Can't wait for a competitor to drive them out of business, as soon as, possible." 
#" Everything is already done, dusted and confirmed with guests and I still get on average of 4 emails afterwards stating my time to respond is running out etc. etc.  Grrrrrrrrr - I am seriously thinking of cancelling my arrangement with airbnb if only I knew how?"

In [52]:
X_valid =  vectorizer.transform([string])
classifier_xgb.predict_proba(X_valid)

array([[0.6229384 , 0.37706158]], dtype=float32)